# Importing ORACC Data from corpus.json
by Niek Veldhuis
UC Berkeley

March 2017

# TODO
* check that COFs are treated properly
* check that lines that continue into the next line (as in bilinguals) are captured completely. Such lines are indicated in the json by the the addition of 'l' (lower case L) to the reference (.ref).
* add definition of fields to list in Introduction

# Note
Currently the code will fetch a large zip file from ORACC, download it, extract certain files from the zip file and parse those. The zip file contains all data that belong to an ORACC project or sub-project. Since one may run this notebook several tines for collecting data from the same project, this is not a good process (the download will take place every time). Move the download process to a separate notebook, preceding the current one.


# Introduction

Purpose of the code is to download [ORACC](http://oracc.org) (Open Richly Annotated Cuneiform Corpus) JSON files that contain textual data and produce a `.csv` file in the directory `data/raw` with the relevant data for use in the phylogenetics project. The JSON files contain all the transliteration and lemmatization data of an ORACC project (metadata are made available in a separate `.json` file). For an introduction to the various ORACC JSON files see the [ORACC Open Data](http://oracc.org/doc/opendata) page.

The resulting data file includes various elements of the ORACC data structure. The current code will output a file with the following fields: 

* id_line
* label
* lemma
* base
* extent
* scope

The fields `extent` and `scope` capture the number of missing lines or columns.

The selection of fields may be adjusted with standard `Pandas` functions.

## Notes
The current version of the script works with the `ijson` library. Documentation for [ijson](https://www.dataquest.io/blog/python-json-tutorial/), unfortunately, is extremely brief. Downloading the `.json` files from the ORACC server will take some time. The rest of the script is reasonably fast.

This notebook is written for **Python 3.5** with **Pandas 0.19** and **ijson 2.3**.

The notebook was written for the [Digital Humanities Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project with Erin Becker of [Data Carpentry](http://www.datacarpentry.org). The particular data selection and data manipulation performed in this notebook are inspired by the needs of that project (for instance, non-Sumerian words are filtered out). It should be fairly easy to adapt the notebook to the purposes of any other project that wishes to use [ORACC](http://oracc.org) data.

## Licensing
This notebook may be downloaded, used and adapted without any restrictions.

In [2]:
import pandas as pd   
import ijson
import urllib.request
import re
import zipfile
import tqdm

ImportError: No module named 'ijson'

# Input List of Text IDs
Identify a list of text IDs (P, Q, and X numbers) in the directory `input`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs, and the filename should end in `.txt`.

In [2]:
filename = input('Filename: ')

Filename: Q01_par.txt


In [4]:
textids = '../data/text_ids/' + filename
with open(textids, 'r') as f:
    pqxnos = f.readlines()
pqxnos = [x.strip() for x in pqxnos]
projects = [x[:-8] for x in pqxnos]
projects = list(set(projects))
pqxnos = [x[-7:] for x in pqxnos]
pqxnos[:5], projects

(['P235800', 'P247532', 'P247533', 'P247526', 'P247541'], ['dcclt'])

# Download `json.zip`
For each project from which files are to be processed download the entire project (all the json files) in `http://oracc.museum.upenn.edu/PROJECT/json.zip`. For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `json.zip` may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. For the chunking code see [this page](https://www.smallsurething.com/how-to-read-a-file-properly-in-python/).

Although downloading the entire zip file is time consuming, it will make processing the individual files much more efficient and the code is less likely to break due to interruption in connectivity.

In [5]:
for project in projects:
    url = "http://oracc.museum.upenn.edu/" + project + "/json.zip"
    file = '../data/json/' + project + '_json.zip'
    response = urllib.request.urlopen(url)
    CHUNK = 16 * 1024
    with open(file, 'wb') as f:
        for chunk in iter(lambda: response.read(CHUNK), b''):
            f.write(chunk)

# Extract
Extract the texts listed in the list of text IDs from the `json.zip`. All files are extracted to a directory called `data/json/corpusjson`. If a list of text IDs has the same P number multiple times (e.g. if editions of the same text exist in multiple projects), the file will be overwritten and only one instance of that P number will be available.

In [7]:
for project in projects:
    zip_file = "../data/json/" + project + "_json.zip"
    target_dir = '../data/json'
    with zipfile.ZipFile(zip_file,"r") as zip_ref:
        for no in pqxnos:
            try:
                file = 'corpusjson/' + no + '.json'
                zip_ref.extract(file, target_dir)
            except:
                print(no + ' is not available')

P247532 is not available
P387682 is not available
P387683 is not available
P247536 is not available
P247534 is not available
P247535 is not available
P247538 is not available
P247552 is not available
P247537 is not available
P247539 is not available
P247528 is not available
P249368 is not available
P225060 is not available
P235796 is not available
P235797 is not available
P235799 is not available
P229115 is not available
P228065 is not available
P229116 is not available
P247530 is not available
P247527 is not available
P247529 is not available
P247531 is not available
P247540 is not available


# Parse
The function `oraccjasonparser()` takes one argument (the **url** of the `.json` file). It looks for the prefix `textid` to retrieve the six-digit P, Q, or X number of the text artifact. Parsing the file sequentially the code looks for the places where a line starts (`'.type' = 'line-start'`) and where a word starts (`'.node' = 'l'`, where `l` is for "lemma"). At each level the code will retrieve the relevant data and create a list where each entry is a dictionary that represents a single word. 

Words not only include lemmatized words, but also unlemmatized and unlemmatizable words (such as breaks).

The dictionary includes the keys `id_line` and `id_word` that allow the user to reassemble words and lines in order.

In [8]:
def oraccjsonparser(file):
    filename = '../data/json/corpusjson/' + file +'.json'
    with open(filename, 'r') as d:
#    d = urllib.request.urlopen(url)
        parser = ijson.parse(d)
        word_l = []
        word_d = {}
        line_start = False
        word_start = False
        nonx = False
        for prefix, event, value in parser:
            if prefix == 'textid':
                id_text = value
#            print("parsing " + value)
            if prefix.endswith('.type'):
                if value == 'line-start':
                    line_start = True
                else:
                    line_start = False
            if line_start:
                if prefix.endswith('.ref') and not word_start:
                    id_line = value # id_line is a reference number for a line
                                # that includes the id_text (e.g. P123456.49)
                if prefix.endswith('.label'):
                    label = value   # label is a human-readable line number of the format
                                # o ii 24' (obverse column 2 line 24')
            if prefix.endswith('node'):
                if value == 'l':
                    word_start = True
                    if not word_d == {}:
                        word_l.append(word_d) # append the previous word to the list
                    word_d = {}               # and start a new dictionary
                    word_d['id_text'] = id_text # provide each word with appropriate 
                    word_d['id_line'] = id_line # text and line-ID
                    word_d['label'] = label     # and the line label.
                else:
                    word_start = False
            if word_start:
                if prefix.endswith('.ref'):
                    word_d['id_word'] = value
                if prefix.endswith('.sig'):
                    word_d['signature'] = value
                if '.f.' in prefix:
                    category = re.sub('.*\.', '', prefix) # get element after the last dot of the prefix
                    word_d[category] = value # copy each element into the dictionary
            if prefix.endswith('.type'):
                if value == 'nonx':
                    nonx = True
                else:
                    nonx = False
            if nonx:                         # this captures so-called $-lines with information
                if prefix.endswith('.ref'):  # about number of broken lines/columns.
                    id_line = value          # $-lines have their own id_line.
                if prefix.endswith('.strict'):
                    if value == '1':           # select only 'strict' $ lines
                        if not word_d == {}:
                            word_l.append(word_d)
                        word_d = {}
                        word_d['id_line'] = id_line
                        word_d['id_text'] = id_text
                    else:
                        nonx = False
                if prefix.endswith('.extent'): # capture the three elements of strict $ lines
                    word_d['extent'] = value   # namely extent, scope, and state.
                if prefix.endswith('.scope'):
                    word_d['scope'] = value
                if prefix.endswith('.state'):
                    word_d['state'] = value

    word_l.append(word_d)  # make sure that the last word is captured, too.
    return(word_l) # return a list of dictionaries, where each entry (dictionary) in
                   # the list represents a word.

# Call the Parser Function for Each Textid

In [9]:
word_l = []
for id_text in tqdm.tqdm(pqxnos):
    try:
        word_l.extend(oraccjsonparser(id_text))
    except:
        print(id_text + ' not available')

 10%|█         | 4/39 [00:00<00:01, 19.91it/s]

P247532 not available
P387682 not available
P387683 not available
P247536 not available
P247534 not available
P247535 not available
P247538 not available
P247552 not available
P247537 not available
P247539 not available
P247528 not available
P249368 not available
P225060 not available
P235796 not available
P235797 not available
P235799 not available
P229115 not available
P228065 not available
P229116 not available
P247530 not available
P247527 not available
P247529 not available
P247531 not available
P247540 not available


100%|██████████| 39/39 [00:00<00:00, 25.32it/s]


# Transform the Data into a DataFrame

If a text has no breakage information in the form of `$ 1 line broken` (etc.) the fields `extent`, `scope`, and `state` do not exist. The fields `extent` and `scope` are referenced in the code below. After creating the dataframe the existence of these two fields is checked - if they do not exist, empty columns are created.

In [10]:
words = pd.DataFrame(word_l)
if not 'extent' in words.columns:
    words['extent'] = ''
if not 'scope' in words.columns:
    words['scope'] = ''
words.head()

,base,cf,cont,epos,extent,form,gw,id_line,id_text,id_word,label,lang,morph,norm,norm0,pos,scope,sense,signature,state
0,muš,muš,NaN,N,NaN,muš,snake,P235800.3,P235800,P235800.3.1,o 1,sux,~,NaN,muš,N,NaN,snake,@dcclt%sux:muš=muš[snake//snake]N'N$muš/muš#~,NaN
1,muš,muš,NaN,N,NaN,muš,snake,P235800.4,P235800,P235800.4.1,o 2,sux,~,NaN,muš,N,NaN,snake,@dcclt%sux:muš=muš[snake//snake]N'N$muš/muš#~,NaN
2,huš,huš,NaN,V/i,NaN,huš,reddish,P235800.4,P235800,P235800.4.2,o 2,sux,~,NaN,huš,V/i,NaN,"(to be) furious, angry","@dcclt%sux:huš=huš[reddish//(to be) furious, a...",NaN
3,muš,muš,NaN,N,NaN,muš,snake,P235800.5,P235800,P235800.5.1,o 3,sux,~,NaN,muš,N,NaN,snake,@dcclt%sux:muš=muš[snake//snake]N'N$muš/muš#~,NaN
4,ušum,ušum,NaN,N,NaN,ušum,snake,P235800.5,P235800,P235800.5.2,o 3,sux,~,NaN,ušum,N,NaN,snake,@dcclt%sux:ušum=ušum[snake//snake]N'N$ušum/ušum#~,NaN


# Remove Spaces and Commas from Guide Word and Sense
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [11]:
words = words.fillna('') # first replace Missing Values by empty string
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* signature (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

# Manipulate
The columns may be manipulated with standard Pandas methods to create the desired output. By way of example, the following code will create a column `lemma` with the format **cf[gw]pos** (for instance **lugal[king]N**). For words that have no lemmatization, `lemma` equals `form`. Only Sumerian words are allowed (and thus `lang` can be omitted) and in addition to the column `lemma` the column `base` is preserved; words that have no lemmatization take `form` as their base. Words and bases are concatenated to lines.

## Remove  non-Sumerian words

In [12]:
lang = ['sux', ''] # note that 'lang' is empty in entries that indicate damage
words = words.loc[words['lang'].str[:3].isin(lang)].reset_index()

## Create Lemma Column and Adjust Base

In [13]:
words['lemma'] = words['cf'] # first element of lemma is the citation form
words['lemma'] = [words['lemma'][i] + '[' + words['gw'][i] 
                     + ']' + words['pos'][i] 
                     if not words['lemma'][i] == '' 
                     else words['form'][i] +'[NA]NA' for i in range(len(words))]
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = [words['base'][i] if not words['base'][i] == '' 
                 or words['label'][i] == '' else words['form'][i] 
                 for i in range(len(words))]
lemmas = words[['lemma', 'base', 'id_text', 'id_line', 'id_word', 'label', 'extent', 'scope']]
lemmas.head()

,lemma,base,id_text,id_line,id_word,label,extent,scope
0,muš[snake]N,muš,P235800,P235800.3,P235800.3.1,o 1,,
1,muš[snake]N,muš,P235800,P235800.4,P235800.4.1,o 2,,
2,huš[reddish]V/i,huš,P235800,P235800.4,P235800.4.2,o 2,,
3,muš[snake]N,muš,P235800,P235800.5,P235800.5.1,o 3,,
4,ušum[snake]N,ušum,P235800,P235800.5,P235800.5.2,o 3,,


## Group by Line

In [14]:
lines = words.groupby([words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'base': ' '.join,
        'extent': ''.join, 
        'scope': ''.join
    }).reset_index()
        

In [15]:
df = lines[['id_line', 'label', 'lemma', 'base', 'extent', 'scope']]
df

,id_line,label,lemma,base,extent,scope
0,P225009.3,o 1,udu[sheep]N urmah[lion]N gu[eat]V/t,udu ur-mah gu₇,,
1,P225009.4,o 2,udu[sheep]N urbara[wolf]N gu[eat]V/t,udu ur-bar-ra gu₇,,
2,P225009.5,o 3,udu[sheep]N diŋir[deity]N gu[eat]V/t,udu diŋir gu₇,,
3,P235800.3,o 1,muš[snake]N,muš,,
4,P235800.4,o 2,muš[snake]N huš[reddish]V/i,muš huš,,
5,P235800.5,o 3,muš[snake]N ušum[snake]N,muš ušum,,
6,P235800.6,o 4,muš[snake]N a[water]N,muš a,,
7,P247526.10,a i 09',udu[sheep]N šu[hand]N ur[roof]N,udu šu ur₃,,
8,P247526.100,b ii 17',guru[shield]N erin[people]N,{kuš}gur₂₁ erin₂,,
9,P247526.101,b ii 18',guru[shield]N x[NA]NA,{kuš}gur₂₁ x,,


# Create fields `id_text` and `line`
The field `id_text` is the P-number of the document and is equal to the first seven characters of `id_line`. The field `line` is an artificial line counter (integer) that is used to keep things in the right order. It derives from `id_line` by keeping eveything after the last dot (note that the regular expression `.+\.` is greedy and will match the longest possible string ending in a dot).

In [16]:
pd.options.mode.chained_assignment = None
df['id_text'] = df['id_line'].str[:7]
df['line'] = [int(re.sub('.+\.', '', line)) for line in df['id_line']] #create a line number for sorting
df = df.sort_values(['id_text', 'line']).reset_index(drop=True)
df

,id_line,label,lemma,base,extent,scope,id_text,line
0,P225009.3,o 1,udu[sheep]N urmah[lion]N gu[eat]V/t,udu ur-mah gu₇,,,P225009,3
1,P225009.4,o 2,udu[sheep]N urbara[wolf]N gu[eat]V/t,udu ur-bar-ra gu₇,,,P225009,4
2,P225009.5,o 3,udu[sheep]N diŋir[deity]N gu[eat]V/t,udu diŋir gu₇,,,P225009,5
3,P235800.3,o 1,muš[snake]N,muš,,,P235800,3
4,P235800.4,o 2,muš[snake]N huš[reddish]V/i,muš huš,,,P235800,4
5,P235800.5,o 3,muš[snake]N ušum[snake]N,muš ušum,,,P235800,5
6,P235800.6,o 4,muš[snake]N a[water]N,muš a,,,P235800,6
7,P247526.3,,,,2,line,P247526,3
8,P247526.4,a i 03',udu[sheep]N kudkudra[disabled]N,udu kud-kud-ra₂,,,P247526,4
9,P247526.5,a i 04',udu[sheep]N niga[fattened]V/i,udu niga,,,P247526,5


## Save in CSV Format

In [17]:
filename = '../data/raw/' + filename[:-4] + '.csv'
print('saving ' + filename)
with open(filename, 'w') as w:
    df.to_csv(w, encoding='utf8')

saving ../data/raw/Q01_par.csv
